```sql
SELECT 
    long_name, /*столбец long_name таблицы teams*/
    home_team_goals, /*столбец home_team_goals таблицы matches*/
    away_team_goals /*столбец away_team_goals таблицы matches*/
FROM
    sql.teams, /*таблица с командами*/
    sql.matches /*таблица с матчами*/
WHERE home_team_api_id = api_id /*условие: home_team_api_id таблицы matches равен api_id таблицы teams*/
```
___

```sql
SELECT 
    long_name, /*столбец long_name таблицы teams*/
    home_team_goals, /*столбец home_team_goals таблицы matches*/
    away_team_goals /*столбец away_team_goals таблицы matches*/
FROM    
    sql.teams /*таблица с командами*/
JOIN sql.matches on home_team_api_id = api_id /*оператор соединения таблиц; таблица matches; условие: home_team_api_id таблицы matches равен api_id таблицы teams*/
```
___

```sql
SELECT
    h.long_name "домашняя команда",
    m.home_team_goals "голы домашней команды",
    m.away_team_goals "голы гостевой команды",
    a.long_name "гостевая команда" 
FROM
    sql.matches m
JOIN sql.teams h ON m.home_team_api_id = h.api_id
JOIN sql.teams a ON m.away_team_api_id = a.api_id
```
___

```sql
SELECT
    m.season,
    t.long_name,
    SUM(m.home_team_goals) + SUM(m.away_team_goals) total_goals
FROM sql.matches m
JOIN sql.teams t ON t.api_id = m.home_team_api_id OR t.api_id = m.away_team_api_id
GROUP BY m.season, t.id
HAVING SUM(m.home_team_goals) + SUM(m.away_team_goals) > 100
```
___

*мы можем вывести сумму голов матча, забитых командами, агрегированную по гостевым командам (совокупное количество голов в матче, забитых обеими командами, суммированное в разрезе гостевых команд)*
```sql
SELECT
    t.long_name,
    SUM(m.home_team_goals) + SUM(m.away_team_goals) match_goals
FROM
    sql.matches m
JOIN sql.teams t ON m.away_team_api_id = t.api_id
GROUP BY t.id
```
___

*вывести таблицу с суммарным количеством забитых голов в матчах по командам и сезонам для команд, в которых суммарное количество голов в матчах сезона больше 100*
```sql
SELECT
    m.season,
    t.long_name,
    SUM(m.home_team_goals) + SUM(m.away_team_goals) total_goals /*функция суммирования; столбец home_team_goals таблицы m; функция суммирования; столбец away_team_goals таблицы m; новое название столбца*/
FROM sql.matches m
JOIN sql.teams t ON t.api_id = m.home_team_api_id OR t.api_id = m.away_team_api_id
GROUP BY m.season, t.id
HAVING SUM(m.home_team_goals) + SUM(m.away_team_goals) > 100
```
___

*Напишите запрос, с помощью которого можно вывести список полных названий команд, сыгравших в гостях 150 и более матчей. Отсортируйте список по названию команды.*
```sql
SELECT
    t.long_name
FROM sql.matches m
JOIN sql.teams t ON t.api_id = m.away_team_api_id
GROUP BY t.id
HAVING COUNT(m.id) >= 150
ORDER BY t.long_name
```
___

JOIN(INNER) пересечение множеств; 

LEFT JOIN; 

RIGHT JOIN; 

FULL OUTER JOIN объединение множеств; 

CROSS JOIN перемножение таблиц, умножение строк головной таблицы на кроссджоин таблицу; 

NATURAL JOIN соединение по колонкам с одинаковым названием
___


***LEFT JOIN***

*выведите список уникальных названий команд, содержащихся в таблице matches. Отсортируйте список в алфавитном порядке*
```sql
SELECT t.long_name
FROM sql.teams t
LEFT JOIN sql.matches m ON t.api_id = m.away_team_api_id OR t.api_id = m.home_team_api_id
WHERE m.id IS NOT NULL
GROUP BY t.long_name
ORDER BY t.long_name
```
___

*напишите запрос, который выведет полное название команды (long_name), количество матчей, в которых участвовала команда, — домашних и гостевых (matches_cnt). Отсортируйте по количеству матчей в порядке возрастания, затем — по названию команды в алфавитном порядке*

```sql
SELECT
    t.long_name long_name,
    COUNT(m.id) matches_cnt
FROM sql.teams t
LEFT JOIN sql.matches m ON t.api_id = m.away_team_api_id OR t.api_id = m.home_team_api_id
GROUP BY t.id
ORDER BY 2, t.long_name
``````
___

***CROSS JOIN***

*все возможные комбинации полных названий команд в матчах*
```sql
SELECT
    DISTINCT /*оператор исключения повторяющихся строк*/
    t1.long_name home_team, /*столбец long_name таблицы t1; новое название*/
    t2.long_name away_team /*столбец long_name таблицы t2; новое название*/
FROM
    sql.teams t1 /*таблица teams с алиасом t1*/
    CROSS JOIN sql.teams t2 /*оператор соединения CROSS JOIN; таблица teams с алиасом t2*/
```
___

*Напишите запрос, который выведет все возможные уникальные комбинации коротких названий домашней команды (home_team) и коротких названий гостевой команды (away_team). Команда не может сама с собой играть, то есть быть и домашней, и одновременно гостевой (в одном и том же матче). Отсортируйте запрос по первому и второму столбцам.*
```sql
SELECT
    DISTINCT
    t1.short_name home_team, 
    t2.short_name away_team
FROM
    sql.teams t1
    CROSS JOIN sql.teams t2
WHERE t1.id != t2.id
ORDER BY 1, 2
```
___

***NATURAL JOIN***

Ключевое слово natural в начале оператора JOIN позволяет не указывать условие соединения таблиц — для соединения будут использованы столбцы с одинаковым названием из этих таблиц.

NATURAL JOIN можно использовать с любыми видами соединений, которые требуют условия соединения:

* NATURAL INNER JOIN (возможна запись NATURAL JOIN);
* NATURAL LEFT JOIN;
* NATURAL RIGHT JOIN;
* NATURAL FULL OUTER JOIN.
___

INTO сохранение в новую таблицу